In [4]:
from py2neo import Graph, Node, Relationship, NodeMatcher

URL = "neo4j://localhost:7687"
AUTH = ("neo4j", "neo4j123456")
graph = Graph(URL, auth=AUTH, name='neo4j')
matcher = NodeMatcher(graph)

In [10]:
# 查询所有标签
query = "CALL db.labels() YIELD label RETURN label"
result = graph.run(query)

# 提取标签并打印
labels = [record["label"] for record in result]
print("Labels in Neo4j:", labels)

Labels in Neo4j: ['Entity', 'Person', 'MagicalCreature', 'Object', 'Place', 'Miscellaneous', 'Event', 'Document', 'Organisation', 'House', 'Spell', 'QuidditchTeam', 'Potion', 'Organization', 'Creature', 'Time', 'Class', 'Position', 'Animal', 'Vehicle', 'Harry Potter', 'Letter', 'Uncle Vernon', 'Dudley', 'Postman', 'Sleeping bag', 'Aunt Petunia', 'Post Office', 'Food Processor', 'Milkman', 'Dairy', 'Minerva McGonagall', 'Hogwarts School of Witchcraft and Wizardry', 'Currency', 'Harry', 'Gringotts', 'Dumbledore', 'Goblins', 'Hogwarts', 'Sport', 'Dursleys', 'Hedwig', 'Hagrid', "King's Cross", 'Guard', 'Group', 'Title', 'Ghost', 'Cat', 'Corridor', 'Plant', 'Team', 'Broomstick', 'Mirror', 'Crowd of people', 'Woman', 'Tall, thin, black haired man', 'Family', 'Document/Artifact', 'Lord Voldemort', 'Muggle', 'Book', 'Mandrake', 'Plant Pot', 'Earmuffs', 'House Elf', 'Location', 'Ministry', 'Dwarf', 'Subject', 'Ron Weasley', 'Hermione Granger', 'George Weasley', 'Dudley Dursley', 'Hogwarts Expre

In [5]:
file_path = 'data/duiqi.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()


In [6]:
lines[:10]

['Mrs Dursley: Petunia=Mrs Dursley=Aunt Petunia\n',
 'Professor McGonagall: Professor McGonagall=McGonagall=Minerva McGonagall\n',
 'Lily: Lily=Mrs Potter=Lily Potter\n',
 'Professor Dumbledore: Professor Dumbledore=Dumbledore=Albus Dumbledore\n',
 'Hagrid: Rubeus Hagrid=Hagrid\n',
 'James Potter: James Potter=Mr Potter\n',
 'Harry Potter: Harry=Harry Potter\n',
 'Dudley Dursley: Dudley Dursley=Dudley\n',
 'Vernon Dursley: Dursley=Mr Dursley=Uncle Vernon\n',
 'Mr and Mrs Dursley: Aunt and uncle=Mr and Mrs Dursley=Dursleys\n']

In [16]:
len_lines = len(lines)-1
for index,line in enumerate(lines):
    print(f'{index}/{len_lines}')
    line = line.strip()
    need_name,merge_str = line.split(': ')
    merge_name_list = merge_str.split('=')
    for merge_name in merge_name_list:
        for label in labels:
            a_node = matcher.match(label, name=need_name).first()
            b_node = matcher.match(label, name=merge_name).first()
            if not a_node and not b_node:
                continue
            if a_node and b_node:
                print(f"Merging {label}: {need_name} and {label}: {merge_name}")
                query = f"""
                        MATCH (a:`{label}` {{name: "{need_name}"}}), (b:`{label}` {{name: "{merge_name}"}})
                        CALL apoc.refactor.mergeNodes([a, b], {{properties: "overwrite", mergeRels: true}}) YIELD node
                        RETURN node
                        """
                print(query)
                graph.run(query)
            elif b_node:
                print(f"rename {label}: {merge_name} to {label}: {need_name}")
                query = f"""
                        MATCH (a:`{label}` {{name: "{merge_name}"}}) set a.name = "{need_name}"
                        """
                print(query)
                graph.run(query)
            else:
                print('未匹配到！')
            print('*****************************************')
        print('=========================================')
print('对齐实体完成！')

0/41
Merging Person: Mrs Dursley and Person: Petunia

                        MATCH (a:`Person` {name: "Mrs Dursley"}), (b:`Person` {name: "Petunia"})
                        CALL apoc.refactor.mergeNodes([a, b], {properties: "overwrite", mergeRels: true}) YIELD node
                        RETURN node
                        
*****************************************
rename Person: Aunt Petunia to Person: Mrs Dursley

                        MATCH (a:`Person` {name: "Aunt Petunia"}) set a.name = "Mrs Dursley"
                        
*****************************************
rename Aunt Petunia: Aunt Petunia to Aunt Petunia: Mrs Dursley

                        MATCH (a:`Aunt Petunia` {name: "Aunt Petunia"}) set a.name = "Mrs Dursley"
                        
*****************************************
1/41
Merging Person: Professor McGonagall and Person: Professor McGonagall

                        MATCH (a:`Person` {name: "Professor McGonagall"}), (b:`Person` {name: "Professor McGona